# Kentro

In [20]:
# Import the CockpitATS client and required modules
import sys
import os
from datetime import datetime
import requests


# Add the parent directory to Python path to import clients module
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from clients.kentro import KentroClient

base_url = os.environ.get("KENTRO_BASE_URL")
username = os.environ.get("KENTRO_USERNAME")
password = os.environ.get("KENTRO_PASSWORD")
test_email = os.environ.get("KENTRO_TEST_EMAIL")

In [21]:
method = "GET"
url = f"{base_url}/candidates" 
response = requests.request(
    method, 
    url, 
    auth=(username, password), # <--- This is required
    headers={"Accept": "application/json"}
)

print(f"Status Code: {response.status_code}")

data = response.json()

Status Code: 200


In [22]:
data

[{'CandidateId': 100000001,
  'CandidateCode': 'TES0000001',
  'FirstName': 'Eddie',
  'Prefix': 'de PvT',
  'Surname': 'Test',
  'candidateExternalIdPartys': [],
  'Gender': 'M',
  'GenderDescription': 'Man',
  'Initial': 'E.',
  'MaritalStatus': '?',
  'MaritalStatusDescription': 'Onbekend',
  'OwnNamePrefix': None,
  'OwnName': None,
  'PartnerSurname': None,
  'DateOfBirth': '1988-05-05',
  'PlaceOfBirth': None,
  'NationalityId': 10062,
  'NationalityDescription': 'Nederlandse',
  'LanguageId': 10001,
  'LanguageDescription': 'Nederlands',
  'PrivateAddress': {'Street': 'Keesomstraat',
   'Addition': None,
   'Number': '28',
   'NumberAddition': '00-A',
   'PostalCode': '6716 AB',
   'City': 'EDE',
   'CountryCode': 'NL',
   'CountryDescription': 'Nederland'},
  'ResidentialAddress': {'Street': 'Keesomstraat',
   'Addition': None,
   'Number': '28',
   'NumberAddition': '00-A',
   'PostalCode': '6716 AB',
   'City': 'EDE',
   'CountryCode': 'NL',
   'CountryDescription': 'Nederlan

In [3]:
for i in data[0].keys():
    print(i)
    

CandidateId
CandidateCode
FirstName
Prefix
Surname
candidateExternalIdPartys
Gender
GenderDescription
Initial
MaritalStatus
MaritalStatusDescription
OwnNamePrefix
OwnName
PartnerSurname
DateOfBirth
PlaceOfBirth
NationalityId
NationalityDescription
LanguageId
LanguageDescription
PrivateAddress
ResidentialAddress
EmailAddress
MobilePhoneNumber
HomePhoneNumber
paymentSettings
DateHired
DateOfLastMediation
LastContact
LastApplicationDate
OriginatesFrom
IsOnlendCandidate
LendingAgencyId
LendingAgencyName
IsSelfEmployed
CandidateSelfEmployedData
HasDrivingLicense
MeansOfTransport
MeansOfTransportDescription
MaximumAvailableHoursWeek
MinimalAvailableHoursWeek
AllowsPersonalDataUsage
AllowsApplicationDataUsage
CurrentStatus
UnsubscribeRequestDate
UnsubscribeRequestBy
IsDeactivated
DeactivationReason
IntermediaryId
BranchId


In [4]:
print(username, base_url, password)

kentro_client = KentroClient()

candidate_id = kentro_client.get_candidate_id_from_email(test_email)
candidate_id

prdmoveapi@timing.nl https://api.flexberichten.nl/api/v1 HoU!M9%B2X&wtv


100000001

In [28]:
type(str(candidate_id))

str

In [29]:
kentro_client.get_payslips(str(candidate_id))


[]

In [6]:
kentro_client.get_reservation_balances(candidate_id)


[]

In [7]:
kentro_client.get_contracts(candidate_id)

[{'ContractId': 48576,
  'Mention': '@C512840',
  'ContractCode': '00000003',
  'CandidateId': 100000001,
  'FlexPhase': 'A',
  'FlexType': 'F',
  'StartDate': '2024-01-01T00:00:00',
  'EndDate': '2024-02-02T00:00:00',
  'ReasonEndLabourRelationship': None,
  'ReasonEndLabourRelationshipDescription': None,
  'CurrentStatus': 'Gesloten',
  'IsFinalized': True,
  'IsApproved': False,
  'Employment': '0',
  'EmploymentDescription': 'Normaal',
  'IsOnCallAgreement': True,
  'IsOpenEnded': False,
  'BusinessUnitId': 10022,
  'BusinessUnitCode': '2',
  'BusinessUnitTaxcategoryId': 10267,
  'CategoryId': 10418,
  'CategoryCode': '221',
  'PaymentSystemId': 10001,
  'FunctionId': 15789,
  'FunctionDescription': 'Tester',
  'AmitCode': 'IT',
  'Wage': 16.89,
  'WageTimeUnit': 'Uur',
  'ReservationMethodId': 10001,
  'HasAgreedNumberOfHours': False,
  'AgreedNumberOfHours': None,
  'AgreedNumberOfHoursPer': None,
  'AgreedNumberOfHoursPerDescription': None,
  'PeriodOfNotice': None,
  'PeriodOfN

# Cockpit API

In [8]:
# Import the CockpitATS client and required modules
import sys
import os
from datetime import datetime

# Add the parent directory to Python path to import clients module
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from clients.cockpit import CockpitATSClient

print("Successfully imported CockpitATSClient")
print(f"Current time: {datetime.now()}")

cockpit_token = os.getenv("COCKPIT_KEY")
cockpit_client = CockpitATSClient("RN-f83d6706-edfb-4b90-a15d-f482ac23f716")

Successfully imported CockpitATSClient
Current time: 2025-11-13 12:38:35.907705


In [9]:
cockpit_client._get_bearer_token()

Error during Cockpit token exchange: 404 Client Error: Not Found for url: https://timing-prelive.recruitnowcockpit.nl/rest/api/v2/candidateprofile/api/authentication/token


HTTPError: 404 Client Error: Not Found for url: https://timing-prelive.recruitnowcockpit.nl/rest/api/v2/candidateprofile/api/authentication/token

In [ ]:
# Test CockpitATSClient (requires identity_provider_token)
print("Testing CockpitATSClient...")

# Example of how to test with a mock token (for demonstration purposes)
try:
    test_token = os.getenv("COCKPIT_KEY")
    cockpit_client = CockpitATSClient(test_token)
    
    # Test get_branch_details method
    print("\nTesting get_branch_details method...")
    try:
        branch_details = cockpit_client.get_branch_details()
        print("Successfully retrieved branch details:")
        print(f"   Response type: {type(branch_details)}")
        if isinstance(branch_details, dict):
            print(f"   Number of fields: {len(branch_details)}")
            print(f"   Fields: {list(branch_details.keys())}")
        else:
            print(f"   Response: {branch_details}")
    except Exception as branch_error:
        print(f"get_branch_details failed: {branch_error}")
        
except Exception as e:
    print(f"CockpitATSClient test failed: {e}")
    print("Make sure you have:")
    print("1. COCKPIT_BASE_URL environment variable set")
    print("2. A valid identity_provider_token (COCKPIT_KEY environment variable)")

Testing CockpitATSClient...

Testing get_branch_details method...
Error during Cockpit token exchange: 401 Client Error: Unauthorized for url: https://timing-prelive.recruitnowcockpit.nl/candidateprofile/api/authentication/token
get_branch_details failed: 401 Client Error: Unauthorized for url: https://timing-prelive.recruitnowcockpit.nl/candidateprofile/api/authentication/token


# Planbition

In [1]:
from langchain_core.tools import tool
from pydantic import BaseModel, Field
from typing import List, Dict, Any
import sys
import os
import requests

# Add the parent directory to Python path to import clients module
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from clients.planbition import PlanbitionClient

# Initialize the client (this will fail if environment variables are not set)
try:
    client = PlanbitionClient()
    print("Successfully initialized PlanbitionClient")
except ValueError as e:
    print(f"Failed to initialize PlanbitionClient: {e}")
    print("Make sure the following environment variables are set:")
    print("- PLANBITION_BASE_URL")
    print("- PLANBITION_KEY") 
    print("- PLANBITION_USERNAME")
    print("- PLANBITION_PASSWORD")
    client = None

Successfully initialized PlanbitionClient


Test endpoints

In [2]:
# Add this temporarily to your client to test
test_employee = client.get_employee_details("RE0036443")
test_employee

{'employee_id': 'RE0036443',
 'employee_number': 'RE0036443',
 'full_name': 'Malgorzata Zwawiak',
 'email': 'gosiak127@poczta.onet.pl',
 'country': 'Nederland',
 'is_active': True}

In [3]:
schedule = client.get_employee_schedule("RE0036443", "2025-05-01", "2025-05-30")
print(f"Found {len(schedule)} schedule items.")


Found 7 schedule items.


In [4]:
schedule

[ScheduleItem(id='DEF894FC-1A82-493D-AE67-7FD0C2E94177_627B699E-0FA6-4153-AC90-C32EBFC230DD_280520250730', employeeNumber='RE0036443', personId='RE0036443', organisationStructureId='Bleckmann Nederland - Nijverheidsstraat', organisationStructureName='Bleckmann Nederland - Nijverheidsstraat', customerId='0FDD57AF-5DA2-432A-B95F-C158CEBC822A', customerName='Bleckmann Nederland - Nijverheidsstraat', customerStructureName='Ruud', shiftId='F40D2C95-433D-4446-A200-E3A6D5E1A215', shiftName='Ruud Reco', shiftQualificationDemandId='627B699E-0FA6-4153-AC90-C32EBFC230DD', shiftQualificationDemandName='Ruud Reco', scheduleEmployeeAdministrativeComponents=[], startTime={'year': 2025, 'month': 5, 'day': 28, 'hour': 7, 'minute': 30, 'second': 0, 'millisecond': 0}, endTime={'year': 2025, 'month': 5, 'day': 28, 'hour': 16, 'minute': 30, 'second': 0, 'millisecond': 0}, plannerRemarkPublic=None, plannerRemarkPrivate=None, employeeRemark=None, iso_start_time='2025-05-28T07:30:00'),
 ScheduleItem(id='DEF89

In [8]:
# Print fields from the data model
for item in schedule:
    print("\n")
    print(f"Shift: {item.shiftName}") 
    print(f"Time: {item.iso_start_time}")
    print(f"ID: {item.customerName}")



Shift: Ruud Reco
Time: 2025-05-28T07:30:00
ID: Bleckmann Nederland - Nijverheidsstraat


Shift: Ruud Reco
Time: 2025-05-27T07:30:00
ID: Bleckmann Nederland - Nijverheidsstraat


Shift: Ruud Reco
Time: 2025-05-23T07:30:00
ID: Bleckmann Nederland - Nijverheidsstraat


Shift: Ruud Reco
Time: 2025-05-22T07:30:00
ID: Bleckmann Nederland - Nijverheidsstraat


Shift: Ruud Reco
Time: 2025-05-21T07:30:00
ID: Bleckmann Nederland - Nijverheidsstraat


Shift: Ruud Reco
Time: 2025-05-20T07:30:00
ID: Bleckmann Nederland - Nijverheidsstraat


Shift: Ruud Reco
Time: 2025-05-16T07:30:00
ID: Bleckmann Nederland - Nijverheidsstraat
